<a href="https://colab.research.google.com/github/bennyb-bio/Group-A-Project/blob/test/covid_19_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd
import os

In [7]:
# Verzeichnis mit den CSV-Dateien
material_path = "/covid19_data"

In [9]:
# SQL-Datenbank, die im Skript 'covid_19_db' erstellt wurde 
database_file = "covid19_data.db"

In [15]:
conn = sqlite3.connect(database_file)
cursor = conn.cursor()

In [47]:
# Mapping-Tabelle von SNOMED nach ICD 
## erstellt von ChatGPT
map_icd = pd.read_csv("map_snomed_icd.csv", sep = ";")
map_icd.head()

,snomed,snomed_desc,icd,icd_desc
0,22298006,Myocardial Infarction,I21.9,"Myocardial infarction, unspecified"
1,25064002,Headache (finding),R51,Headache
2,36955009,Loss of taste (finding),R43.1,Ageusia
3,40095003,Injury of kidney (disorder),S37.9,"Injury of kidney, unspecified"
4,43724002,Chill (finding),R68.83,Chills


# Mapping

## Tabelle conditions

Tabelle conditions enthält Diagnosen in Form von SNOMED-CT-Codes. Mapping auf ICD-Klassifikation

In [53]:
# conditions table 
# Alle distinkten SNOMED-Codes aus Tabelle conditions laden und mit Mapping-Tabelle verknüpfen
query = """
SELECT DISTINCT code as snomed, description as snomed_desc
FROM conditions
ORDER BY code desc
"""

df = pd.read_sql_query(query, conn)
df_mapped = df.merge(map_icd, how="left", on = ["snomed", "snomed_desc"])
df_mapped

,snomed,snomed_desc,icd,icd_desc
0,132281000119108,Acute deep venous thrombosis (disorder),I82.409,Acute venous embolism and thrombosis of unspec...
1,840544004,Suspected COVID-19,U07.2,"COVID-19, virus not identified"
2,840539006,COVID-19,U07.1,COVID-19
3,770349000,Sepsis caused by virus (disorder),A41.9,"Sepsis, unspecified"
4,706870000,Acute pulmonary embolism (disorder),I26.9,"Pulmonary embolism, unspecified"
5,429007001,History of cardiac arrest (situation),Z86.73,Personal history of ischemic heart disease
6,422587007,Nausea (finding),R11.0,Nausea
7,410429000,Cardiac Arrest,I46.9,"Cardiac arrest, unspecified"
8,399211009,History of myocardial infarction (situation),Z86.73,Personal history of ischemic heart disease
9,389087006,Hypoxemia (disorder),R09.02,Hypoxemia


In [85]:
print(f"Tabelle conditions:\nFür {df_mapped.icd.notna().sum()} von {len(df_mapped)} SNOMED-CT-Codes konnte ein passender ICD-Code gefunden werden")



Tabelle conditions:
Für 41 von 41 SNOMED-CT-Codes konnte ein passender ICD-Code gefunden werden


In [ ]:
# Verbindung zur Datenbank lösen
conn.close()